In [76]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train = tf.keras.utils.normalize(x_train ,axis = 1)
x_test = tf.keras.utils.normalize(x_test ,axis = 1)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation = 'relu'))
model.add(tf.keras.layers.Dense(128, activation = 'relu'))
model.add(tf.keras.layers.Dense(128, activation = 'softmax'))

model.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])
model.fit(x_train,y_train,epochs = 5)

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3193 - accuracy: 0.9107
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1169 - accuracy: 0.9643
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0814 - accuracy: 0.9752
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0590 - accuracy: 0.9817
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0438 - accuracy: 0.9857


In [77]:
val_loss , val_acc = model.evaluate(x_test,y_test)
print(val_loss,val_acc)

313/313 [==============================] - 0s 2ms/step - loss: 0.0961 - accuracy: 0.9728
0.09612475335597992 0.9728000164031982


In [78]:
model.save('model.h5')

In [79]:
new_model = tf.keras.models.load_model('model.h5')

In [80]:
predictions = new_model.predict([x_test])

In [81]:
print(predictions)

[[1.3371786e-10 1.3388457e-09 3.7075745e-07 ... 1.8630124e-17
  6.0563762e-17 2.8107878e-17]
 [1.2261947e-11 2.1328832e-04 9.9976438e-01 ... 7.1876014e-19
  2.8855378e-17 5.2986785e-17]
 [4.4562245e-07 9.9948597e-01 4.5597503e-06 ... 6.1456201e-11
  3.0144040e-11 3.5967965e-11]
 ...
 [3.9720475e-12 1.2636488e-07 3.7650317e-11 ... 4.3121863e-15
  9.6706963e-15 4.4076486e-15]
 [5.4535558e-08 8.3337763e-09 1.5091613e-08 ... 2.5782336e-13
  6.8463692e-13 9.8100022e-13]
 [5.5579746e-10 3.2797793e-09 9.0555021e-09 ... 3.6426331e-17
  5.6418662e-16 2.1101585e-16]]


In [82]:
import numpy as np

print(np.argmax(predictions[8]))

5


In [88]:
import matplotlib.pyplot as plt


In [89]:
scores = model.evaluate(x_test,y_test,verbose = 0)

In [92]:
from keras.models import load_model
model = load_model('Downloads//model.h5')

In [48]:
import os
import sys
import PIL 
import cv2
import glob
from tkinter import *
from tkinter.ttk import *
from PIL import Image,ImageDraw,ImageGrab


In [49]:
root = Tk()
root.resizable(0,0)
root.title('HANDWRITTEN DIGIT RECOGNITION GUI APP')

lastx,lasty = None,None
image_number = 0

def clear_widget():
    global cv
  #to clear canvas
    cv.delete("all")

def activate_event(event):
    global lastx,lasty
    cv.bind('<B1-Motion',draw_lines)
    lastx,lasty = event.x,event.y

def draw_lines(event):
    global lastx,lasty
    x,y = event.x,event.y
    cv.create_line((lastx, lasty,x,y),width = 8,fill = 'black',capstyle = ROUND, smooth = TRUE,splinesteps=12)
    lastx,lasty = x,y
    
cv = Canvas(root, width = 640, height = 480 ,bg = 'white')
cv.grid(row = 0,column = 0,pady = 2,sticky=W,columnspan = 2)

#Tkinter provides a powerful mechanism to let you deal with event yourself
cv.bind('<Button-1>',activate_event)

#Add buttons and labels
btn_save = Button(text = 'Recognize Digit',command = Recognize_Digit)
btn_save.grid(row=2,column=0,pady=1,padx=1)
button_clear = Button(text='clear widget',command = clear_widget)
button_clear.grid(row=2,column=1,pady=1,padx=1)

def Recognize_Digit():
    global image_number
    predictions = []
    percentage = []
  # image_no = 0
    filename = f'image_{image_number}.png'
    widget = cv

  #get the widget co-ordinates
    x = root.winfo_rootx()+widget.winfo_x()
    y = root.winfo_rooty()+widget.winfo_y()
    x1 = x+widget.winfo_width()
    y1= y+widget.winfo_height()
  #grab the image,crop it accordingly to my requirement and saved it in png format
    ImageGrab.grab().crop((x,y,x1,y1)).save(filename)
    
image = cv2.imread(filename,cv2.IMREAD_COLOR)
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
ret,th = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+CV2.THRESH_OTSU)
contours = cv2.findContours(th,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[0] 

for cnt in contours:
  x,y,w,h = cv2.boundingRect(cnt)
  cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),1)
  top = int(0.05*th.shape[0])
  bottom = top
  left = int(0.05*th.shape[1])
  right = left
  th_up = cv2.copyMakeBorder(th,top,bottom,left,right,cv2.BORDER_REPLICATE)
  roi = th[y-top:y+h+bottom,x-left:x+w+right]
  image = cv2.resize(roi,(28,28),interpolation = cv2.INTER_AREA)
  img = img.reshape(1,28,28,1)
  img = img/255.0
  pred = model.predict([img])[0]
  final_pred = np.argmax(pred)
  data = str(final_pre)+''+str(int(max(pred)*100))+'%'
  font = cv2.FONT_HERSHEY_SIMPLEX
  fontscale = 0.5
  color = (255,0,0)
  thickness =1
  cv2.putText(image,data,(x,y-5),font,fontScale,color,thickness)

#showing predicted result on new window
cv2.imshow('image',image)
cv2.waitKey(0)  
root.mainloop()